In [1]:
import pandas as pd
import numpy as np
import time

In [2]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
stop_words = nltk.corpus.stopwords.words("english")
for word in ['what', 'how', 'where', 'who', 'which'] :
    stop_words.append(word)
from string import punctuation

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [3]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import jaccard_score
from sklearn.linear_model import LogisticRegression

In [4]:
from sklearn.multioutput import MultiOutputClassifier

In [5]:
file = open("/content/top_10_tags.txt", "r")
top_10_tags = file.read()
top_10_tags = list(top_10_tags.split('\n')[:-1])
file.close()

In [6]:
data = pd.read_csv("/content/data.csv", engine = 'python', encoding='utf-8', error_bad_lines=False)

<ipython-input-6-12374c331878>:1: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  data = pd.read_csv("/content/data.csv", engine = 'python', encoding='utf-8', error_bad_lines=False)


In [7]:
text = data['Title']
text_spl = text.sample(frac = 0.25).reset_index()
text_spl.head()

,index,Title
0,17865,How to import self-signed SSL certificate to V...
1,8427,Getting Windows OS version programmatically
2,8859,Rounded Corners on UIImage
3,3048,Android Gauge Animation Question
4,39135,pandas - change df.index from float64 to unico...


In [8]:
def preprocess(text) :

    """" Nettoyage du texte :
    passage au minuscule
    suppression du code éventuel du texte que l'on stocke dans une variable 'code'
    suppression et du contenu des balises autres que p (script, alt, ...)
    suppression des balises html
    conservation des textes labellisés par les top 10 tags uniquement
    suppression de la ponctuation, des chiffres,
    et des stopwords
    lemmatisation par spaCy """
    
    text = text.lower()
    
    for i in range(1, len(text)) :
        if text[i-1] == 'c' and text[i] == '#' :
            text = text.replace(text[i], 'sharp')
    
    token_list = nltk.word_tokenize(text)
    
    new_text = []
    
    for token in token_list :
        if token in top_10_tags :
            new_text.append(token)
        elif token not in stop_words :
            for char in token :
                if char in punctuation or char.isdigit() :
                    token = token.replace(char, '')
            new_text.append(token)
    
    lem = nltk.stem.WordNetLemmatizer()
    
    for token in new_text :
        if nltk.pos_tag([token])[0][1].startswith('V') :
            index = new_text.index(token)
            token_lem = lem.lemmatize(token, pos = 'v')
            new_text[index] = new_text[index].replace(token, token_lem)
            
    new_text = ' '.join(new_text)

    return new_text

In [9]:
print("Textes bruts :")
print("")
print(text_spl.loc[:11, 'Title'])
print("---------------------------------------")
print("Textes nettoyés :")
print("")
print(text_spl.loc[:11, 'Title'].apply(preprocess))

Textes bruts :

0     How to import self-signed SSL certificate to V...
1           Getting Windows OS version programmatically
2                            Rounded Corners on UIImage
3                      Android Gauge Animation Question
4     pandas - change df.index from float64 to unico...
5               Rvalues, lvalues and formal definitions
6     Recursion on reactive streams with Project Rea...
7                      Cross-thread operation not valid
8     Can I serve JSPs from inside a JAR in lib, or ...
9               Getting current device language in iOS?
10    React native deploying on iOS device stuck on ...
11    COM Interop: What should I do to make a C# pro...
Name: Title, dtype: object
---------------------------------------
Textes nettoyés :

0     import selfsigned ssl certificate volley android 
1               get windows os version programmatically
2                                 round corners uiimage
3                      android gauge animation question
4 

In [10]:
%%time
text_clean = text_spl['Title'].apply(preprocess)

CPU times: user 19.2 s, sys: 1.02 s, total: 20.2 s
Wall time: 30.4 s


In [11]:
text_spl['Title_clean'] = text_clean

In [12]:
data = pd.merge(data.iloc[text_spl['index']], text_spl)[['Title', 'Title_clean', 'Tags']]
data.head(3)

,Title,Title_clean,Tags
0,How to import self-signed SSL certificate to V...,import selfsigned ssl certificate volley android,['android']
1,Getting Windows OS version programmatically,get windows os version programmatically,"['c#', 'windows']"
2,Rounded Corners on UIImage,round corners uiimage,"['ios', 'iphone', 'objective-c']"


In [13]:
import tensorflow_hub as hub

embed = hub.load("https://tfhub.dev/google/universal-sentence-encoder/4")

In [14]:
def feature_USE_fct(sentences, b_size) :
    batch_size = b_size
    time1 = time.time()

    for step in range(len(sentences)//batch_size) :
        idx = step*batch_size
        feat = embed(sentences[idx:idx+batch_size])

        if step ==0 :
            features = feat
        else :
            features = np.concatenate((features,feat))

    time2 = np.round(time.time() - time1,0)
    return features

In [15]:
batch_size = 10
sentences = data['Title_clean'].to_list()

In [16]:
features_USE = feature_USE_fct(sentences, batch_size)

In [17]:
data = data.join(pd.DataFrame(features_USE))

In [18]:
for tag in top_10_tags :
    data['is' + tag] = 0
    index = 0
    for doc_tag in data['Tags'] :
        if not pd.isnull(doc_tag) :
          if tag in doc_tag :
            data.loc[index, 'is' + tag] = 1
        index += 1

In [19]:
data

,Title,Title_clean,Tags,0,1,2,3,4,5,6,...,isc#,isjava,isjavascript,ispython,isc++,isios,isandroid,is.net,ishtml,isphp
0,How to import self-signed SSL certificate to V...,import selfsigned ssl certificate volley android,['android'],-0.049040,-0.017037,-0.062048,0.047834,0.012593,0.067784,-0.033927,...,0,0,0,0,0,0,1,0,0,0
1,Getting Windows OS version programmatically,get windows os version programmatically,"['c#', 'windows']",-0.070347,0.005683,-0.038418,-0.017030,0.024135,0.029856,-0.005335,...,1,0,0,0,0,0,0,0,0,0
2,Rounded Corners on UIImage,round corners uiimage,"['ios', 'iphone', 'objective-c']",-0.056571,-0.002925,0.008424,0.023003,0.062388,-0.068799,-0.000654,...,0,0,0,0,0,1,0,0,0,0
3,Android Gauge Animation Question,android gauge animation question,['android'],-0.028710,0.004977,0.024333,-0.023212,-0.023399,-0.067385,-0.003584,...,0,0,0,0,0,0,1,0,0,0
4,pandas - change df.index from float64 to unico...,pandas change dfindex float unicode string,['python'],-0.043451,-0.069809,0.026422,0.066117,0.058382,-0.048583,-0.055396,...,0,0,0,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10573,DateTimeOffset.Now in T-SQL,datetimeoffsetnow tsql,"['sql', 'sql-server']",NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0
10574,Which GoF Design pattern will be changed or in...,gof design pattern change influence introducti...,['java'],NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,1,0,0,0,0,0,0,0,0
10575,How to define a more aggressive timeout for Ht...,define aggressive timeout httpwebrequest,"['c#', '.net']",NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1,0,0,0,0,0,0,1,0,0
10576,iOS 7 app crashes when changing calendar privacy,ios app crashes change calendar privacy,"['ios', 'xcode']",NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,1,0,0,0,0


In [20]:
X = data.iloc[:,3:-10]
y = data.iloc[:,-10:]

In [21]:
X = X[~X[0].isna()]
y = y.iloc[X.index]

In [22]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.33, random_state = 42)

In [23]:
%%time
mclr = MultiOutputClassifier(LogisticRegression(max_iter = 1000)).fit(X_train, y_train)

CPU times: user 5.09 s, sys: 1.26 s, total: 6.35 s
Wall time: 3.43 s


In [24]:
mclr.score(X_train, y_train)

0.5445558536929812

In [25]:
mclr.score(X_test, y_test)

0.5176268271711092

In [26]:
jaccard_score(y_test, mclr.predict(X_test), average = 'micro')

0.4907732864674868

In [27]:
def clean_output(result) :

  output_tags_list = []

  for i in range(len(result)) :
    if result[i] == 1 :
      output_tags_list.append("#" + top_10_tags[i])

  return output_tags_list

In [28]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import FunctionTransformer

In [29]:
transformer = FunctionTransformer(feature_USE_fct, kw_args={'b_size':10}, validate = True)

In [30]:
transformer.fit(X_train, y_train)
mclr.fit(X_train, y_train)

MultiOutputClassifier(estimator=LogisticRegression(max_iter=1000))

In [31]:
pipe = Pipeline([('USE', transformer), ('MultiLogReg', mclr)], verbose = True)

In [32]:
from joblib import dump

In [33]:
dump(pipe, 'trained_use_logreg.joblib')

['trained_use_logreg.joblib']